# Table of Contents
1. [sizeof operator](#sizeof-operator)
2. [Signed Integers](#signed-integers)
3. [Unsigned Integers](#unsigned-integers)
4. [Fixed Width Integers](#fixed-width-integers)
5. [sizeof_t](#sizeof_t)
6. [Scientific Notation](#scientific-notation)
7. [Floating point numbers](#floating-point-numbers)
8. [Boolean Values](#boolean-values)
9. [Introduction to if statements](#introduction-to-if-statements)
10. [Chars](#chars)
11. [Type Conversion and static_cast](#type-conversion-and-static_cast)



## sizeof operator
- Takes **type** or **variable** as input.
- Returns size of the object in **bytes**.

In [1]:
#include <iomanip> // for std::setw (which sets the width of the subsequent output)
#include <iostream>
#include <climits> // for CHAR_BIT

In [2]:
int main()
{
    std::cout << "A byte is " << CHAR_BIT << " bits\n\n";

    std::cout << std::left; // left justify output

    std::cout << std::setw(16) << "bool:" << sizeof(bool) << " bytes\n";
    std::cout << std::setw(16) << "char:" << sizeof(char) << " bytes\n";
    std::cout << std::setw(16) << "short:" << sizeof(short) << " bytes\n";
    std::cout << std::setw(16) << "int:" << sizeof(int) << " bytes\n";
    std::cout << std::setw(16) << "long:" << sizeof(long) << " bytes\n";
    std::cout << std::setw(16) << "long long:" << sizeof(long long) << " bytes\n";
    std::cout << std::setw(16) << "float:" << sizeof(float) << " bytes\n";
    std::cout << std::setw(16) << "double:" << sizeof(double) << " bytes\n";
    std::cout << std::setw(16) << "long double:" << sizeof(long double) << " bytes\n";

    return 0;
}

// Manually call main in notebook
main(); 

A byte is 8 bits

bool:           1 bytes
char:           1 bytes
short:          2 bytes
int:            4 bytes
long:           8 bytes
long long:      8 bytes
float:          4 bytes
double:         8 bytes
long double:    16 bytes


- Results may vary based on compiler, computer arcitecture, OS, compilation settings (32-bit vs 64-bit) etc...
- Trying to use `sizeof` on incomplete type (such as `void`) will result in compilation error

---

## Signed Integers

- Signed integers are integers that can only hold positive and negative whole numbers, including 0.

In [3]:
// Defining signed integers
short s;      // prefer "short" instead of "short int"
int i;
long l;       // prefer "long" instead of "long int"
long long ll; // prefer "long long" instead of "long long int"


// However signed keyword should not be used, as it is redundant, since integers are signed by default.
signed short ss;
signed int si;
signed long sl;
signed long long sll;

### Overflow
- **Integer overflow / Arithmetic overflow:** If an arithmetic operation (such as addition or multiplication) attempts to create a value outside the range that can be represented, this is called integer overflow (or arithmetic overflow). For signed integers, integer overflow will result in undefined behavior.



In [4]:
int main()
{
    // assume 4 byte integers
    int x { 2'147'483'647 }; // the maximum value of a 4-byte signed integer
    std::cout << x << '\n';

    x = x + 1; // integer overflow, undefined behavior
    std::cout << x << '\n';

    return 0;
}

// Manually call main in notebook
main(); 

2147483647
-2147483648


---
## Unsigned Integers

- Unsigned integers are integers that can only hold non-negative whole numbers.
- An n-bit unsigned variable has a range of 0 to (2^n)-1.
- It is suggested not to use unsigned integers unless it is required / common to do so due to the overflow behaviour it has.

In [5]:
// Defining unsigned integers
unsigned short us;
unsigned int ui;
unsigned long ul;
unsigned long long ull;

### Overflow
- “wraps around” / “modulo wrapping”

In [6]:
int main()
{
    unsigned short x{ 65535 }; // largest 16-bit unsigned value possible
    std::cout << "x was: " << x << '\n';

    x = 65536; // 65536 is out of our range, so we get modulo wrap-around
    std::cout << "x is now: " << x << '\n';

    x = 65537; // 65537 is out of our range, so we get modulo wrap-around
    std::cout << "x is now: " << x << '\n';

    std::cout << "----------------------------------" << '\n';

    unsigned short y{ 0 }; // smallest 2-byte unsigned value possible
    std::cout << "y was: " << y << '\n';

    y = -1; // -1 is out of our range, so we get modulo wrap-around
    std::cout << "y is now: " << y << '\n';

    y = -2; // -2 is out of our range, so we get modulo wrap-around
    std::cout << "y is now: " << y << '\n';

    return 0;
}


main();

input_line_12:5:9: warning: implicit conversion from 'int' to 'unsigned short' changes value from 65536 to 0 [-Wconstant-conversion]
    x = 65536; // 65536 is out of our range, so we get modulo wrap-around
      ~ ^~~~~
input_line_12:7:9: warning: implicit conversion from 'int' to 'unsigned short' changes value from 65537 to 1 [-Wconstant-conversion]
    x = 65537; // 65537 is out of our range, so we get modulo wrap-around
      ~ ^~~~~


x was: 65535
x is now: 0
x is now: 1
----------------------------------
y was: 0
y is now: 65535
y is now: 65534


- Second, and more insidiously, unexpected behavior can result when you mix signed and unsigned integers. In C++, if a mathematical operation (e.g. arithmetic or comparison) has one signed integer and one unsigned integer, the signed integer will usually be converted to an unsigned integer. And the result will thus be unsigned. For example:

- You have 4 bits to hold the number
  - signed `-1` -> `1 0 0 1`. First bit(MSB - Most Significant Bit) is reserved for sign (+ / -).
  - Comparision b/w signed and unsigned => signed is converted into unsigned
  - `1 0 0 1` -> converted to value is `9` (which looks like wrap around / modulo wrapping)

In [7]:
// assume int is 4 bytes
int main()
{
    signed int s { -1 };
    unsigned int u { 1 };

    if (s < u) // -1 is implicitly converted to 4294967295, and 4294967295 < 1 is false
        std::cout << "-1 is less than 1\n";
    else
        std::cout << "1 is less than -1\n"; // this statement executes

    return 0;
}

main();

1 is less than -1


---

## Fixed Width Integers

- `C++11` provides an alternate set of integer types that are guaranteed to be the same size on any architecture. 
- The fixed-width integers are defined in the `<cstdint>` header. 
- `std::int#_t` (signed) or `std::uint#_t` (unsigned) where `#` can be [8, 16, 32, 64] bits i.e., [1, 2, 4, 8] bytes
- **Warning:** `std::int8_t` and `std::uint8_t` typically behave like chars

### Issues
- The fixed-width integers are not guaranteed to be defined on all architectures.
- If you use a fixed-width integer, it may be slower than a wider type on some architectures.


### Fast and least integral types

- To help address the above downsides, C++ also defines two alternative sets of integers that are guaranteed to exist.
- The fast types (`std::int_fast#_t` and `std::uint_fast#_t`) provide the fastest signed/unsigned integer type with a width of at least `#` bits (where # = 8, 16, 32, or 64). 
- The least types (`std::int_least#_t` and `std::uint_least#_t`) provide the smallest signed/unsigned integer type with a width of at least `#` bits (where # = 8, 16, 32, or 64).



In [8]:
#include <cstdint> // for fixed-width integers
#include <iostream>

int main()
{
    std::int32_t x { 32767 }; // x is always a 32-bit integer
    std::cout << sizeof(x) << " bytes.\n";

    std::cout << "----------------------------------" << '\n';
    
    std::int8_t y { 65 };   // initialize 8-bit integral type with value 65
    std::cout << y << '\n'; // You're probably expecting this to print 65

    return 0;
}


main();

4 bytes.
----------------------------------
A


In [9]:
#include <cstdint> // for fast and least types
#include <iostream>

int main()
{
	std::cout << "least 8:  " << sizeof(std::int_least8_t)  * 8 << " bits\n";
	std::cout << "least 16: " << sizeof(std::int_least16_t) * 8 << " bits\n";
	std::cout << "least 32: " << sizeof(std::int_least32_t) * 8 << " bits\n";
	std::cout << '\n';
	std::cout << "fast 8:  "  << sizeof(std::int_fast8_t)   * 8 << " bits\n";
	std::cout << "fast 16: "  << sizeof(std::int_fast16_t)  * 8 << " bits\n";
	std::cout << "fast 32: "  << sizeof(std::int_fast32_t)  * 8 << " bits\n";

	return 0;
}


main();

least 8:  8 bits
least 16: 16 bits
least 32: 32 bits

fast 8:  8 bits
fast 16: 64 bits
fast 32: 64 bits


---
### sizeof_t

- We can infer that operator `sizeof` returns an integer value -- but what integral type is that return value? An int? A short? The answer is that `sizeof` returns a value of type `std::size_t`.
- `std::size_t` is an alias for an implementation-defined unsigned integral type. 
- In other words, the compiler decides if `std::size_t` is an unsigned int, an unsigned long, an unsigned long long, etc…
- std::size_t is actually a `typedef`.

In [10]:
#include <cstddef>  // for std::size_t
#include <iostream>

int main()
{
    int x { 5 };
    std::size_t s { sizeof(x) }; // sizeof returns a value of type std::size_t, so that should be the type of s
    std::cout << s << '\n';

    // Amusingly, we can use the sizeof operator (which returns a value of type std::size_t) to ask for the size of std::size_t itself:
    std::cout << sizeof(std::size_t) << '\n';

    return 0;
}


main();

4
8


---
## Scientific Notation

- Because it can be hard to type or display exponents in C++, we use the letter ‘e’ (or sometimes ‘E’) to represent the “times 10 to the power of” part of the equation. For example, 1.2 x 10⁴ would be written as 1.2e4, and 5.9722 x 10²⁴ would be written as 5.9722e24.
- For numbers smaller than 1, the exponent can be negative. The number 5e-2 is equivalent to 5 * 10⁻², which is 5 / 10², or 0.05. The mass of an electron is 9.1093837e-31 kg.

In [11]:
float x =  3.14e5;
float x1 =  3.14e6;
double y = 3.14e-4;
double y1 = 3.14e-5;

std::cout << x << '\n' << x1 << '\n';
std::cout << y << '\n' << y1 << '\n';

314000
3.14e+06
0.000314
3.14e-05


---
### Floating point numbers

- C++ has three fundamental floating point data types: a single-precision `float` (4 bytes), a double-precision `double` (8 bytes), and an extended-precision `long double` (8, 12 or 16 bytes). As with integers, C++ does not define the actual size of these types.
- On modern architectures, floating-point types are conventionally implemented using one of the floating-point formats defined in the IEEE 754 standard (see https://en.wikipedia.org/wiki/IEEE_754). As a result, float is almost always 4 bytes, and double is almost always 8 bytes.
- On the other hand, long double is a strange type. On different platforms, its size can vary between 8 and 16 bytes, and it may or may not use an IEEE 754 compliant format. We recommend avoiding long double.
- Note that by default, floating point literals default to type double. An `f` suffix is used to denote a literal of type float.


In [12]:
#include <iostream>
#include <limits>

int main()
{
    std::cout << std::boolalpha; // print bool as true or false rather than 1 or 0
    std::cout << "float: " << std::numeric_limits<float>::is_iec559 << '\n';
    std::cout << "double: " << std::numeric_limits<double>::is_iec559 << '\n';
    std::cout << "long double: " << std::numeric_limits<long double>::is_iec559 << '\n'; // Can vary by platform
}


main();

float: true
double: true
long double: true


In [13]:
// Floating point variables and literals
float f; // Precision: 6-9 significant digits, typically 7
double d; // Precision: 15-18 significant digits, typically 16
long double ld; 

int a { 5 };      // 5 means integer
double b { 5.0 }; // 5.0 is a floating point literal (no suffix means double type by `default`)
float c { 5.0f }; // 5.0 is a floating point literal, f suffix means float type


// Printing floating point numbers
std::cout << 5.0 << '\n'; // By default, std::cout will not print the fractional part of a number if the fractional part is 0.
std::cout << 6.7f << '\n';
std::cout << 9876543.21 << '\n'; // Printed in scientific notation

5
6.7
9.87654e+06


@0x77b3705fcd20

In [14]:
/*
When outputting floating point numbers, std::cout has a default precision of 6 i.e., 
it assumes all floating point variables are only significant to 6 digits (the minimum precision of a float), 
and hence it will truncate anything after that.
*/

std::cout << 9.87654321f << '\n';
std::cout << 987.654321f << '\n';
std::cout << 987654.321f << '\n';
std::cout << 9876543.21f << '\n';
std::cout << 0.0000987654321f << '\n';

9.87654
987.654
987654
9.87654e+06
9.87654e-05


### Output Manipulator

- We can override the default precision that std::cout shows by using an `output manipulator` function named `std::setprecision()`
- Defined in the `iomanip` header.

In [15]:
#include <iomanip> // for output manipulator std::setprecision()
#include <iostream>

int main()
{
    std::cout << std::setprecision(17); // show 17 digits of precision
    std::cout << 3.33333333333333333333333333333333333333f <<'\n'; // f suffix means float
    std::cout << 3.33333333333333333333333333333333333333 << '\n'; // no suffix means double

    return 0;
}


/*
Because we set the precision to 17 digits using std::setprecision(), each of the above numbers is printed with 17 digits. 
But, as you can see, the numbers certainly aren’t precise to 17 digits! And because floats are less precise than doubles, the float has more error.
*/
main();

3.3333332538604736
3.3333333333333335


In [16]:
#include <iomanip> // for std::setprecision()
#include <iostream>

int main()
{
    float f { 123456789.0f }; // f has 10 significant digits
    std::cout << std::setprecision(9); // to show 9 digits in f
    std::cout << f << '\n';

    return 0;
}


/*
The value 123456789.0 has 10 significant digits, but float values typically have 7 digits of precision (and the result of 123456792 is precise only to 7 significant digits).
We lost some precision! When precision is lost because a number can’t be stored precisely, this is called a rounding error.
*/
main();

123456792


### NaN and Inf

IEEE 754 compatible formats additionally support some special values:
- **Inf**, which represents infinity. Inf is signed, and can be positive (+Inf) or negative (-Inf).
- **NaN**, which stands for “Not a Number”. There are several different kinds of NaN.




In [17]:
#include <iostream>

int main()
{
    double zero { 0.0 };

    double posinf { 5.0 / zero }; // positive infinity
    std::cout << posinf << '\n';

    double neginf { -5.0 / zero }; // negative infinity
    std::cout << neginf << '\n';

    double z1 { 0.0 / posinf }; // positive zero
    std::cout << z1 << '\n';

    double z2 { -0.0 / posinf }; // negative zero
    std::cout << z2 << '\n';

    double nan { zero / zero }; // not a number (mathematically invalid)
    std::cout << nan << '\n';

    return 0;
}


main();

inf
-inf
0
-0
-nan


---
## Boolean Values

- https://www.learncpp.com/cpp-tutorial/boolean-values/

---
## Introduction to if statements

- https://www.learncpp.com/cpp-tutorial/introduction-to-if-statements/

---
## Chars

- The char data type is an integral type, meaning the underlying value is stored as an integer. Similar to how a Boolean value 0 is interpreted as false and non-zero is interpreted as true, the integer stored by a char variable are intepreted as an ASCII character.
- `ASCII` stands for `American Standard Code for Information Interchange`, and it defines a particular way to represent English characters (plus a few other symbols) as numbers between 0 and 127 (called an ASCII code or code point). For example, ASCII code 97 is interpreted as the character ‘a’.




- https://www.learncpp.com/cpp-tutorial/chars/

In [18]:
char ch1{ 'a' }; // initialize with code point for 'a' (stored as integer 97) (preferred)
char ch2{ 97 }; // initialize with integer 97 ('a') (not preferred)
std::cout << ch1 << '\n' << ch2 << '\n';


char ch3{5}; // initialize with integer 5 (stored as integer 5 - ascii code)
char ch4{'5'}; // initialize with code point for '5' (stored as integer 53 - ascii code)
std::cout << ch3 << '\n' << ch4 << '\n';


a
a

5


In [19]:
#include <iostream>
#include <sstream> // For istringstream


int main()
{
    // Simulated input
    std::istringstream input("abcd"); 
    std::cin.rdbuf(input.rdbuf()); // Redirect std::cin to use input


    std::cout << "Input a keyboard character: "; // assume the user enters "abcd" (without quotes)

    char ch{};
    std::cin >> ch; // ch = 'a', "bcd" is left queued.
    std::cout << "You entered: " << ch << '\n';

    // Note: The following cin doesn't ask the user for input, it grabs queued input!
    std::cin >> ch; // ch = 'b', "cd" is left queued.
    std::cout << "You entered: " << ch << '\n';

    return 0;
}


main();

Input a keyboard character: You entered: a
You entered: b


In [20]:
// Extracting whitespace characters
#include <iostream>
#include <sstream> // For istringstream


int main()
{
    // Simulated input
    std::istringstream input("a b"); 
    std::cin.rdbuf(input.rdbuf()); // Redirect std::cin to use input

    
    std::cout << "Input a keyboard character: "; // assume the user enters "a b" (without quotes)

    char ch{};
    std::cin >> ch; // extracts a, leaves " b\n" in stream
    std::cout << "You entered: " << ch << '\n';

    std::cin >> ch; // skips leading whitespace (the space), extracts b, leaves "\n" in stream
    std::cout << "You entered: " << ch << '\n';

    return 0;
}


main();

Input a keyboard character: You entered: a
You entered: b


In [21]:
// cin.get() to perform whitespace extraction
#include <iostream>
#include <sstream> // For istringstream

int main()
{
    // Simulated input
    std::istringstream input("a b"); 
    std::cin.rdbuf(input.rdbuf()); // Redirect std::cin to use input


    std::cout << "Input a keyboard character: "; // assume the user enters "a b" (without quotes)

    char ch{};
    std::cin.get(ch); // extracts a, leaves " b\n" in stream
    std::cout << "You entered: " << ch << '\n';

    std::cin.get(ch); // extracts space, leaves "b\n" in stream
    std::cout << "You entered: " << ch << '\n';

    return 0;
}


main();

Input a keyboard character: You entered: a
You entered:  


- `char16_t` and `char32_t` were added to C++11 to provide explicit support for 16-bit and 32-bit Unicode characters. 
- `char8_t` was added in C++20 to provide support for 8-bit Unicode (UTF-8).
- `wchar_t` should be avoided in almost all cases (except when interfacing with the Windows API), as its size is implementation-defined.

---

## Type Conversion and static_cast

- The process of converting a value from one type to another type is called type conversion. 
- In most cases, C++ will allow us to convert values of one fundamental type to another fundamental type.
- When the compiler does type conversion on our behalf without us explicitly asking, we call this **implicit type conversion**.

### Type conversion produces a new value

- Even though it is called a conversion, a type conversion does not actually change the value or type of the value being converted. Instead, the value to be converted is used as input, and the conversion results in a new value of the target type.

### Explicit type conversion via the static_cast operator

- To perform an explicit type conversion, in most cases we’ll use the static_cast operator.
> `static_cast<new_type>(expression)`

In [22]:
// Implicit Type Conversion
#include <iostream>

void print(double x) // print takes a double parameter
{
	std::cout << x << '\n';
}


// int main () { ... }
// By default, floating point values whose decimal part is 0 print without the decimal places (e.g. 5.0 prints as 5).
print(5); // what happens when we pass an int value?


5


In [23]:
// Explicit Type Conversion
#include <iostream>

void printInt(int x) // print now takes an int parameter
{
	std::cout << x << '\n';
}


// int main () { ... }
printInt(5.5); // warning: we're passing in a double value
printInt(static_cast<int>(5.5)); // explicitly convert double value 5.5 to an int


input_line_48:7:11: warning: implicit conversion from 'double' to 'int' changes value from 5.5 to 5 [-Wliteral-conversion]
 printInt(5.5); // warning: we're passing in a double value
 ~~~~~~~~ ^~~


5
5


- https://www.learncpp.com/cpp-tutorial/introduction-to-type-conversion-and-static_cast/
  - Using static_cast to convert char to int
  - Sign Conversions using static_cast
  - std::int8_t and std::uint8_t likely behave like chars instead of integers